# OpenAI Parameters

# Overview

https://github.com/tamasma/master-ia-tajamar/blob/main/05_OpenAI_parameters.ipynb

When making requests to OpenAI models, several parameters can be used to control the behavior and output of the model. \
Understanding these parameters helps in fine-tuning the responses to meet specific requirements, whether for generating text, answering questions, or any other use case.

For more detailed examples, refer to the official documentation [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [ ]:
!pip install openai

In [ ]:
!pip install tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00


In [ ]:
import os

# Definir las variables de entorno dentro de Google Collab
os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_KEY"

In [ ]:
import re
import requests
import sys
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-02-15-preview"
)

CHAT_COMPLETIONS_MODEL = "gpt-4o-mini"
SEED = 123

# Parameter: max_tokens
**Description**: The maximum number of tokens to generate in the completion. \
**Default**: 16 \
**Example**: max_tokens=50

The token count of your prompt plus max_tokens can't exceed the model's context length. \
Most models have a context length of 2048 tokens (except for the newest models, which support 4096). Please refer to documentation.

In [ ]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [16, 32, 60, 100]
for penalty in penalties:
    print(f"Max Tokens: {penalty}\n")
    print(call_openai_with_max_tokens(penalty))
    print("\n" + "-"*80 + "\n")

Max Tokens: 16

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations

--------------------------------------------------------------------------------

Max Tokens: 32

The best pet can vary depending on individual preferences, lifestyles, and needs. Here are a few popular options and some of their benefits:

1. **Dogs**

--------------------------------------------------------------------------------

Max Tokens: 60

The best pet can vary greatly depending on individual preferences, lifestyle, and living situation. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered the best pets for active individuals or families.

2. **Cats**: Cats are generally more independent

--------------------------------------------------------------------------------

Max Tokens: 100

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Some people mig

# Parameter: temperature

**Description**: Controls the randomness of the output. Lower values make the output more deterministic, while higher values increase randomness. \
**Value Range**: 0 to 1 \
**Default Value**: 1 \
**Example**: temperature=0.7

Higher values means the model will take more risks. \
Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.

---
**NOTE**: We generally recommend altering this or top_p but not both.


In [ ]:
def call_openai(num_times, prompt, temperature=0.75, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [ ]:
# Without seed and temperature, the response is different each time
call_openai(10, 'The best pet is a ')

The best pet can vary greatly depending on individual preferences, lifestyles, and needs. Some popular options include:

- **Dogs**: Known for their loyalty and companionship, dogs can be great for active individuals or families who enjoy spending time outdoors.
- **Cats**: Cats are often more independent than dogs
The best pet really depends on individual preferences, lifestyle, and needs. For some people, dogs are the best pets because they're loyal, affectionate, and great companions. Others might prefer cats for their independence and low maintenance. Some people may enjoy the company of small animals like rabbits, hamsters, or
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals and families. They require regular exercise and social interaction.

2. **Cats**: Cats
The best pet can vary greatly depending on in

In [ ]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, 'The best pet is a ', temperature = 0, use_seed=True)

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered "man's best friend." They require regular exercise and social interaction.

2. **Cats**:
The best pet can vary greatly depending on individual preferences, lifestyles, and

# Parameter: n
**Description**: Specifies the number of completions to generate for each prompt. \
**Default Value**: 1 \
**Example**: n = 3

---
**Note**: Because this parameter generates many completions, it can quickly consume your token quota. Use carefully and ensure that you have reasonable settings for max_tokens and stop.

In [ ]:
response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = 0.8,
                    n=3
            )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet is often considered to be one that fits well with your lifestyle, preferences, and living situation. For some, dogs are the best pets due to their loyalty, companionship, and ability to bond closely with humans. Others might prefer cats for their independence and lower maintenance needs. 

If you
1 The best pet can vary greatly depending on a person's lifestyle, preferences, and living situation. Here are a few options, each with its own advantages:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals and families. They require regular exercise, training, and
2 The best pet is often subjective and depends on individual preferences, lifestyles, and circumstances. Here are some popular options and what makes them great:

1. **Dogs**: Known for their loyalty and companionship. They can be great for active individuals or families, as they often enjoy outdoor activities and playtime


# Parameter: presence_penalty
**Description**: Penalizes new tokens based on whether they appear in the text so far, encouraging the model to use new tokens. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: presence_penalty=0.5

In [ ]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    presence_penalty= presence_penalty
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

Presence Penalty: 0

The best pet can vary greatly depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are often considered one of the best pets. They require regular exercise, training, and social interaction.

2.

--------------------------------------------------------------------------------

Presence Penalty: 0.5

The best pet can vary widely depending on an individual's lifestyle, preferences, and needs. Here are some popular options:

1. **Dog**: Often called "man's best friend," dogs are loyal companions and great for active individuals or families. They require regular exercise and social interaction.

2. **Cat

--------------------------------------------------------------------------------

Presence Penalty: 1.0

The best pet can vary depending on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for th

# Parameter: frequency_penalty
**Description**: Penalizes new tokens based on their existing frequency in the text so far, reducing the likelihood of repeating the same line verbatim. \
**Value Range**: -2.0 to 2.0 \
**Default Value**: 0 \
**Example**: frequency_penalty=0.5

#### Use cases to explore
1. **Compare Responses** \
Generate multiple completions to compare and choose the best response for your use case.

2. **Increase Diversity** \
Use multiple completions to get a variety of responses, which is useful in creative applications.

3. **Enhance Robustness** \
Generate multiple responses to ensure consistency and accuracy across different completions.

#### Best Practices
1. **Optimize Prompt Length** \
Keep your prompts concise but informative to ensure the model has enough context.

2. **Adjust Temperature and Top_p** \
Use these parameters to balance between deterministic and creative responses.

3. **Monitor Token Usage** \
Be mindful of the max_tokens parameter to manage costs and response length.

4. **Use Stopping Sequences** \
Define stopping sequences to control where the model should stop generating text, ensuring the output is within the desired context.

5. **Generate Multiple Completions** \
Use the n parameter to generate multiple completions and select the best one for your needs.

## Exercise 1: Exploring 'top_p' Parameter
### Description:
- The 'top_p' parameter controls nucleus sampling. Lower values make the output more focused by limiting the pool of tokens.

### Task:
- Write a function that calls the model with different 'top_p' values and observe how the output changes.


In [ ]:
def call_openai_with_top_p(top_p):
    response = client.chat.completions.create(
          model= CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Describe the benefits of exercise."}],
                    max_tokens=60,
                    top_p= top_p
    )
    return response.choices[0].message.content

# Try with different values of top_p
top_p_values = [0.1, 0.3, 0.7, 1.0]
for value in top_p_values:
    print(f"Top P: {value}\n")
    print(call_openai_with_top_p(value))
    print("\n" + "-"*80 + "\n")


Top P: 0.1

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.3

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits
1. **Improved Cardiovascular Health**: Regular exercise strengthens the heart, improves circulation, and helps lower blood pressure and cholesterol levels.
2. **Weight Management

--------------------------------------------------------------------------------

Top P: 0.7

Exercise offers a wide range of benefits for both physical and mental health. Here are some of the key advantages:

### Physical Health Benefits:
1. 

## Exercise 2: Frequency Penalty Exploration
### Description:
- This exercise will explore how the 'frequency_penalty' parameter affects the model's tendency to repeat itself.

### Task:
- Generate completions using different frequency_penalty values and note the diversity in responses.



In [ ]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Tell me a fun fact about space."}],
                    max_tokens=60,
                    frequency_penalty= frequency_penalty
    )
    return response.choices[0].message.content

penalties = [-1.0, 0.0, 1.0, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")


Frequency Penalty: -1.0

Here's a fun fact about space: There are more stars in the universe than there are grains of sand on all the Earth's beaches! Estimates suggest there are about 100 billion to 200 billion galaxies in the observable universe, each containing millions to trillions of stars. This means the total number of stars

--------------------------------------------------------------------------------

Frequency Penalty: 0.0

A fun fact about space is that a day on Venus is longer than a year on Venus! A single rotation on Venus takes about 243 Earth days, while its orbit around the Sun takes only about 225 Earth days. This means that Venus has extremely slow and prolonged days compared to its year!

--------------------------------------------------------------------------------

Frequency Penalty: 1.0

A fun fact about space is that there are more stars in the universe than there are grains of sand on all of Earth's beaches! Astronomers estimate that there are about 100 bi

## Exercise 3: Multi-Completion with 'n' Parameter
### Description:
- This exercise will help students generate multiple completions for the same prompt and compare the results.

### Task:
- Generate 3 completions for the same prompt by adjusting the 'n' parameter.


In [ ]:
response = client.chat.completions.create(
                model= CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "The best pet is a"}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = 0.8,
                    n=6
            )

for index, c in enumerate(response.choices):
    print(index, c.message.content)

0 The best pet can vary greatly depending on personal preferences, lifestyles, and living situations. Here are some popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise, training, and social interaction.

2. **Cats
1 The best pet can vary greatly depending on a person's lifestyle, preferences, and living situation. Here are a few options, each with its own advantages:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals and families. They require regular exercise, training, and
2 The best pet is often subjective and depends on individual preferences, lifestyles, and living situations. Here are a few popular options:

1. **Dogs**: Known for their loyalty and companionship, dogs are great for active individuals or families. They require regular exercise and training.

2. **Cats**:
3 The best pet can vary greatly depending on individual preference

## Exercise 4: Temperature vs. Deterministic Output
### Description:
- Adjust the 'temperature' parameter to explore the balance between creative and deterministic responses.

### Task:
- Generate completions with different temperature values and analyze the variance in output.


In [ ]:
def call_openai_with_temperature(temperature):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "Suggest a name for a new tech startup."}],
                    max_tokens=30,
                    temperature= temperature
    )
    return response.choices[0].message.content

temperatures = [0.0, 0.3, 0.7, 1.0]
for temp in temperatures:
    print(f"Temperature: {temp}\n")
    print(call_openai_with_temperature(temp))
    print("\n" + "-"*80 + "\n")

Temperature: 0.0

Sure! Here are some suggestions for a tech startup name:

1. **TechNova**
2. **InnoSphere**
3. **ByteBridge

--------------------------------------------------------------------------------

Temperature: 0.3

Sure! Here are some suggestions for a tech startup name:

1. **TechNova**
2. **InnoSphere**
3. **ByteBridge

--------------------------------------------------------------------------------

Temperature: 0.7

Sure! Here are several suggestions for a tech startup name:

1. **InnoVate**
2. **TechNexus**
3. **

--------------------------------------------------------------------------------

Temperature: 1.0

Sure! Here are a few suggestions for tech startup names:

1. **TechNest**
2. **InnoVibe**
3. **Next

--------------------------------------------------------------------------------

